<a href="https://colab.research.google.com/github/moexalgo/moexalgo.github.io/blob/main/examples/library-examples/websocket/streams/securities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install moexalgo
#!pip install matplotlib

import pandas as pd
from moexalgo.beta import issplus

# Данные для входа
ws_login = 'ваш_логин'
ws_passcode = 'ваш_пароль'

ticker = 'SBER'

async def websocket_securities():
    url = 'wss://iss.moex.com/infocx/v3/websocket'
    credentials = issplus.Credentials('passport', ws_login, ws_passcode)
    
    df_snapshot = None
    count = 0
    
    async with issplus.connect(url, credentials) as client:
        # Подписка на рыночные данные
        destination = 'MXSE.securities'
        selector = f'TICKER="MXSE.TQBR.{ticker}"'
        
        subscription = await client.subscribe(destination, selector)
        print(f"Подписка ID: {subscription.id}\n")
        
        async for data in subscription:
            msg_type = data['properties']['type']
            
            # Первое сообщение (snapshot) - полный снимок данных
            if msg_type == 'snapshot':              
                df_snapshot = pd.DataFrame(data['data'], columns=data['columns'])
                
                cols = list(df_snapshot.columns)
                print(f"Доступные поля ({len(cols)}):")
                for i in range(0, len(cols), 5):
                    print("  " + ", ".join(cols[i:i+5]))

                print(f"\nСнэпшот (snapshot):")
                print(df_snapshot[['TICKER', 'LAST', 'BID', 'OFFER', 'VOLTODAY', 'VALTODAY', 'NUMTRADES']].T)
                print()
                
            # Последующие сообщения (updates) - только изменившиеся поля
            elif msg_type == 'updates':
                count += 1
                print(f"Обновление (updates) #{count}")
                
                df_update = pd.DataFrame(data['data'], columns=data['columns'])
                
                print(df_update)
                
                # Обновляем значения снэпшота (snapshot)
                if df_snapshot is not None:
                    for col in df_update.columns:
                        if col in df_snapshot.columns:
                            df_snapshot.at[0, col] = df_update.at[0, col]
                
                print()
            
            # Остановка после пяти итераций (для примера)
            if count >= 5:
                break
    
    return df_snapshot


securities_snapshot = await websocket_securities()

Subscription ID: afefd79d-bb0b-43c0-8889-206d6d75019c

Доступные поля (41):
  TICKER, CAPTION, LAST, CHANGE, BID
  OFFER, HIGH, LOW, HIGHBID, LOWOFFER
  BIDDEPTHT, OFFERDEPTHT, VOLTODAY, VALTODAY, NUMTRADES
  LCLOSEPRICE, LCURRENTPRICE, PREVPRICE, WAPRICE, PREVWAPRICE
  PRICEMINUSPREVWAPRICE, QTY, VALUE, BASEPRICE, MARKETPRICE
  CURRENCYID, SETTLECODE, SETTLEDATE1, SETTLEDATE2, LOTSIZE
  FACEUNIT, FACEVALUE, ISSUESIZE, ADMITTEDQUOTE, TRADINGSTATUS
  STATUS, DPVALINDICATORBUY, DPVALINDICATORSELL, TIME, MINSTEP
  PRICEMINSTEP

Снэпшот (snapshot):
                        0
TICKER     MXSE.TQBR.SBER
LAST          [306.13, 2]
BID           [306.12, 2]
OFFER         [306.13, 2]
VOLTODAY     [1322700, 0]
VALTODAY        405375659
NUMTRADES           10774

Обновление (updates) #1
           TICKER         LAST      CHANGE  BIDDEPTHT  OFFERDEPTHT  \
0  MXSE.TQBR.SBER  [306.12, 2]  [-0.37, 2]    1685771      3001280   

       VOLTODAY   VALTODAY  NUMTRADES PRICEMINUSPREVWAPRICE  QTY  \
0  [132

In [ ]:
# Финальный снэпшот (snapshot)
securities_snapshot.T

,0
TICKER,MXSE.TQBR.SBER
CAPTION,Sberbank
LAST,"[306.12, 2]"
CHANGE,"[-0.37, 2]"
BID,"[306.12, 2]"
OFFER,"[306.13, 2]"
HIGH,"[307.0, 2]"
LOW,"[305.89, 2]"
HIGHBID,"[310.0, 2]"
LOWOFFER,"[295.0, 2]"
